### 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be `language`
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [1]:
# imports
import pyspark
import pandas as pd

# create spark environment
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [10]:
# create pandas dataframe
p_df = pd.DataFrame({'language':['python', 'sql', 'html', 'ruby', 'c', 'scala']})
p_df

,language
0,python
1,sql
2,html
3,ruby
4,c
5,scala


In [11]:
# create spark dataframe
df = spark.createDataFrame(p_df)

In [9]:
# print schema
df.printSchema()

root
 |-- language: string (nullable = true)



In [14]:
# output shape
print("DataFrame shape: ", df.count(), " x ", len(df.columns))

DataFrame shape:  6  x  1


In [13]:
# view first 5 records
df.show(5)

+--------+
|language|
+--------+
|  python|
|     sql|
|    html|
|    ruby|
|       c|
+--------+
only showing top 5 rows

